In [4]:
import pandas as pd

all_games_dataset = pd.read_csv('all_games.csv')

all_games_dataset['release_date'] = pd.to_datetime(all_games_dataset['release_date'])
all_games_dataset.rename(columns={'name': 'Title'}, inplace=True)

all_games_dataset.head()

,Title,platform,release_date,summary,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,1998-11-23,"As a young boy, Link is tricked by Ganondorf, ...",99,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,2000-09-20,As most major publishers' development efforts ...,98,7.4
2,Grand Theft Auto IV,PlayStation 3,2008-04-29,[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.7
3,SoulCalibur,Dreamcast,1999-09-08,"This is a tale of souls and swords, transcendi...",98,8.4
4,Grand Theft Auto IV,Xbox 360,2008-04-29,[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,7.9


In [5]:
import requests

def get_wikipedia_extract(game):
    url = 'https://en.wikipedia.org/w/api.php'
    params = {
        'action': 'query',
        'format': 'json',
        'titles': game,
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
    }
 
    response = requests.get(url, params=params)
    data = response.json()
    if int(next(iter(data['query']['pages'].keys()))) < 0:
        return ''
    page_extract = next(iter(data['query']['pages'].values()))['extract']
    return page_extract

In [26]:
from bs4 import BeautifulSoup

def get_from_infobox(game): 
    url = 'https://en.wikipedia.org/w/api.php'
    params = {
        'action': 'parse',
        'page': game,
        'format': 'json',
        'prop': 'text',
        'redirects': ''
    }
 
    response = requests.get(url, params=params)
    data = response.json()
    raw_html = data['parse']['text']['*']
    return raw_html

def get_genre(game): 
    raw_html = get_from_infobox(game)
    soup = BeautifulSoup(raw_html, 'html.parser')
    tbody = soup.find('tbody')
    genreElement = tbody.find('a', {'title': 'Video game genre'})
    genre = genreElement.find_parent().find_parent().find('td').find('a').text
    return genre

In [27]:
get_genre('Tony Hawk\'s Pro Skater 2')
get_genre('The Legend of Zelda: Ocarina of Time')
get_genre('SoulCalibur')

Sports

Action-adventure

Fighting



In [ ]:
wikipedia_extracts = []

for game in all_games_dataset['Title']:
    wikipedia_extracts.append(get_wikipedia_extract(game))

all_games_dataset['Wikipedia'] = wikipedia_extracts
all_games_dataset.head()

In [3]:
all_games_dataset.to_csv('all_games_clean.csv')
all_games_dataset.head()

,Title,platform,release_date,summary,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,1998-11-23,"As a young boy, Link is tricked by Ganondorf, ...",99,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,2000-09-20,As most major publishers' development efforts ...,98,7.4
2,Grand Theft Auto IV,PlayStation 3,2008-04-29,[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.7
3,SoulCalibur,Dreamcast,1999-09-08,"This is a tale of souls and swords, transcendi...",98,8.4
4,Grand Theft Auto IV,Xbox 360,2008-04-29,[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,7.9
